In [23]:
import os
import pandas as pd
import json
import numpy as np
import pickle
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer
from datasets import load_from_disk, load_metric, Dataset, load_dataset

from captum.concept import TCAV, Concept
from captum.concept._utils.data_iterator import dataset_to_dataloader, CustomIterableDataset


In [21]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
checkpoint = '/work3/s174498/final/linear_head/checkpoint-1500' #finetuning-sentiment-model-test-head3/checkpoint-500'

model = RobertaForSequenceClassification.from_pretrained(checkpoint,output_hidden_states = True,return_dict = True,)
#model.eval()

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at /work3/s174498/final/linear_head/checkpoint-1500 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
# tokenizer
tokenizer_checkpoint = RobertaTokenizer.from_pretrained(checkpoint) 
#tokenizer_pretrained = RobertaTokenizer.from_pretrained('roberta-base')

In [9]:
# load
datadir = '/work3/s174498/sst2_dataset/'
test_0_dataset = load_from_disk(datadir + 'test_0_dataset')
test_1_dataset = load_from_disk(datadir + 'test_1_dataset')

In [6]:
#tokenizer_checkpoint.encode(test_0_dataset['sentence'][0])

In [7]:
model.roberta.encoder.layer[0].attention

RobertaAttention(
  (self): RobertaSelfAttention(
    (query): Linear(in_features=768, out_features=768, bias=True)
    (key): Linear(in_features=768, out_features=768, bias=True)
    (value): Linear(in_features=768, out_features=768, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (output): RobertaSelfOutput(
    (dense): Linear(in_features=768, out_features=768, bias=True)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
)

In [55]:
tcav = TCAV(model, layers=['roberta.encoder.layer.0'])

/zhome/94/5/127021/miniconda3/envs/roberta_env/lib/python3.9/site-packages/captum/concept/_utils/classifier.py:130: UserWarning: Using default classifier for TCAV which keeps input both train and test datasets in the memory. Consider defining your own classifier that doesn't rely heavily on memory, for large number of concepts, by extending `Classifer` abstract class
  warnings.warn(


In [10]:
# Prepare the text inputs for the model
def preprocess_function(examples):
    return tokenizer_checkpoint(examples["sentence"], truncation=True)

tokenized_test = test_0_dataset.map(preprocess_function, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [11]:
N = 0
for i in range(len(test_0_dataset['sentence'])):
    j = len(test_0_dataset['sentence'][i])
    if j > N :
        N = j



In [60]:
def get_tensor_from_filename(filename):
    #datadir = '/work3/s174498/sst2_dataset/'
    ds = load_from_disk(filename)
    const_len = N
    ds_text = ds['sentence']
    text_idx = []
    for i in range(len(ds_text)):
        text_idx.append(tokenizer_checkpoint.tokenize(ds_text[i]))
        text_idx[i] = tokenizer_checkpoint.encode(text_idx[i])
        text_idx[i] += [1] * max(0, const_len - len(text_idx[i]))
    text_indices = torch.tensor(text_idx, device=device)
    print(text_indices)
    yield text_indices

## Hent concept data

In [75]:
def assemble_concept(name, id, concepts_path="/work3/s174498/sst2_dataset/"):
    concept_path = os.path.join(concepts_path, name) + "/"
    #print(concept_path)
    dataset = CustomIterableDataset(get_tensor_from_filename, concept_path)
    #print(dataset.)
    concept_iter = dataset_to_dataloader(dataset)
    #print(concept_iter)

    return Concept(id=id, name=name, data_iter=concept_iter)

In [76]:
pos_concept = assemble_concept("positive", 0, concepts_path="/work3/a174498/sst2_dataset/")
pos_concept2 = assemble_concept("positive", 2, concepts_path="/work3/a174498/sst2_dataset/")

neg_concept = assemble_concept("negative", 1, concepts_path="/work3/a174498/sst2_dataset/")

In [77]:
experimental_sets=[[pos_concept, neg_concept],[ pos_concept2,neg_concept]]

### Input target text
**Should be converted to tensor**

In [78]:
input_text = test_1_dataset['sentence'][:10]

In [79]:
def covert_text_to_tensor(input_texts):
    input_tensors = []
    for input_text in input_texts:
        const_len = N
        #text_idx = tokenizer_checkpoint.tokenize(input_text)
        text_idx = tokenizer_checkpoint.encode(input_text)
        text_idx += [1] * max(0, const_len - len(text_idx))
        input_tensor = torch.tensor(text_idx, device=device).unsqueeze(0)
        input_tensors.append(input_tensor)
    return torch.cat(input_tensors)

In [80]:
input_text_indices = covert_text_to_tensor(input_text)
input_text_indices 

tensor([[    0,  8585,    16,  ...,     1,     1,     1],
        [    0, 12963,   114,  ...,     1,     1,     1],
        [    0,  9178, 38651,  ...,     1,     1,     1],
        ...,
        [    0,   102, 17758,  ...,     1,     1,     1],
        [    0, 11990,    57,  ...,     1,     1,     1],
        [    0, 38597,  1461,  ...,     1,     1,     1]])

In [81]:
tcav = TCAV(model, layers=['roberta.encoder.layer.0'])

In [82]:
tcav.interpret(input_text_indices , experimental_sets=experimental_sets)

FileNotFoundError: Directory /work3/a174498/sst2_dataset/positive/ not found

In [57]:
model.roberta.encoder#.layer[0].attention

RobertaEncoder(
  (layer): ModuleList(
    (0): RobertaLayer(
      (attention): RobertaAttention(
        (self): RobertaSelfAttention(
          (query): Linear(in_features=768, out_features=768, bias=True)
          (key): Linear(in_features=768, out_features=768, bias=True)
          (value): Linear(in_features=768, out_features=768, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (output): RobertaSelfOutput(
          (dense): Linear(in_features=768, out_features=768, bias=True)
          (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (intermediate): RobertaIntermediate(
        (dense): Linear(in_features=768, out_features=3072, bias=True)
        (intermediate_act_fn): GELUActivation()
      )
      (output): RobertaOutput(
        (dense): Linear(in_features=3072, out_features=768, bias=True)
        (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise